In [1]:
#Upload VLE dataset
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'OUData/studentVle.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

  code_module code_presentation  id_student  id_site  date  sum_click
0         AAA             2013J       28400   546652   -10          4
1         AAA             2013J       28400   546652   -10          1
2         AAA             2013J       28400   546652   -10          1
3         AAA             2013J       28400   546614   -10         11
4         AAA             2013J       28400   546714   -10          1


In [2]:
#Let's aggregate the sum_click to get total daily clicks and also ignore different types of sites
# List of columns to group by. This should include all columns except 'sum_click' and 'id_site'
columns_to_group_by = ['code_module', 'code_presentation', 'id_student','date'] 

# Group by the specified columns and sum the 'sum_click' for each group
df_grouped = df.groupby(columns_to_group_by).agg(total_daily_clicks=('sum_click', 'sum')).reset_index()
df_grouped.head(100)

,code_module,code_presentation,id_student,date,total_daily_clicks
0,AAA,2013J,11391,-5,98
1,AAA,2013J,11391,0,49
2,AAA,2013J,11391,1,127
3,AAA,2013J,11391,2,4
4,AAA,2013J,11391,6,3
...,...,...,...,...,...
95,AAA,2013J,28400,150,3
96,AAA,2013J,28400,154,27
97,AAA,2013J,28400,155,1
98,AAA,2013J,28400,156,25


In [3]:
import numpy as np
#create function that returns df with all dates included and 0 for total_daily_clicks for missing dates
def complete_df(code_module,code_presentation,id_student):
    #filter by course, presentation, and student
    filtered_df = df_grouped[(df_grouped['code_module']==code_module) & (df_grouped['code_presentation']==code_presentation) & (df_grouped['id_student']==id_student)]

    #For missing dates, we will make total_daily_clicks to be 0
  
    # Step 1: Find min and max date values
    min_day = filtered_df['date'].min()
    max_day = filtered_df['date'].max()

    # Step 2: Generate complete range of days
    all_days = np.arange(min_day, max_day + 1)  # +1 to include the last day

    # Step 3: Create a DataFrame with all days
    df_all_days = pd.DataFrame({'date': all_days, 'code_module': code_module, 'code_presentation': code_presentation, 'id_student': id_student})
    df_all_days = df_all_days.assign(total_daily_clicks=0)

    # Merge to update total_daily_clicks where data exists
    df_complete = pd.merge(df_all_days, filtered_df[['date', 'total_daily_clicks']], on='date', how='left', suffixes=('', '_update'))
    df_complete['total_daily_clicks'] = df_complete['total_daily_clicks_update'].fillna(df_complete['total_daily_clicks'])
    df_complete.drop(columns='total_daily_clicks_update', inplace=True)

    return df_complete

In [4]:
#Load StudentInfo dataset that has the outcomes of each student for a given module and presentation
file_path = 'OUData/studentInfo.csv'
df_student_info = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df_student_info.head())

  code_module code_presentation  id_student gender                region  \
0         AAA             2013J       11391      M   East Anglian Region   
1         AAA             2013J       28400      F              Scotland   
2         AAA             2013J       30268      F  North Western Region   
3         AAA             2013J       31604      F     South East Region   
4         AAA             2013J       32885      F  West Midlands Region   

       highest_education imd_band age_band  num_of_prev_attempts  \
0       HE Qualification  90-100%     55<=                     0   
1       HE Qualification   20-30%    35-55                     0   
2  A Level or Equivalent   30-40%    35-55                     0   
3  A Level or Equivalent   50-60%    35-55                     0   
4     Lower Than A Level   50-60%     0-35                     0   

   studied_credits disability final_result  
0              240          N         Pass  
1               60          N         Pass  

In [5]:
#We want to create interaction metrics from click data that only go out to n days after start date.
#We can find the best n that preserves accuracy while being as low as possible so that we can give the warning signal as
#early as possible.

#Create a new dataframe that has student id, mean of total daily clicks, ratio of zero-click days, and standard deviation of total daily clicks
def df_interaction_n(code_module,code_presentation,n):

    list_student_ids = df_grouped[(df_grouped['code_module']==code_module) & (df_grouped['code_presentation']==code_presentation)]['id_student'].unique()
    # Initialize an empty list to store the data
    data = []

    # Loop through each student ID
    for student_id in list_student_ids:
        # Filter the DataFrame for the current student ID
        filtered_df = complete_df(code_module,code_presentation,student_id)
        filtered_n_df = filtered_df[filtered_df['date'] <= n]
        # Handling cases where filtered_n_df might be empty
        if not filtered_n_df.empty:
            mean_clicks = filtered_n_df['total_daily_clicks'].mean()
            zero_click_days = (filtered_n_df['total_daily_clicks'] == 0).sum()
            total_days = filtered_n_df.shape[0]
            stdv = filtered_n_df['total_daily_clicks'].std()
            total_clicks = filtered_n_df['total_daily_clicks'].sum()
            
           
        else:
            # Default values when there's no data within the given period
            mean_clicks = 0
            zero_click_days = 0
            total_days = 0
            zero_day_ratio = 0  # Adjusted to avoid division by zero
            stdv = 0
            total_clicks = 0

        # Compute the ratio of zero-click days, ensuring not to divide by zero
        zero_day_ratio = zero_click_days / total_days if total_days > 0 else 0
        # Handling NaN for stdv explicitly if stdv calculation results in NaN
        stdv = 0 if pd.isna(stdv) else stdv
         #final result
        filtered_series = df_student_info[(df_student_info['code_module']==code_module )& (df_student_info['code_presentation']==code_presentation)&(df_student_info['id_student']==student_id)]['final_result']
        final_result = filtered_series.iloc[0] if not filtered_series.empty else None

#         # Calculate the mean of the 'total_clicks' column for this student
#         mean_clicks = filtered_df[filtered_df['date']<=n]['total_daily_clicks'].mean()

#         # Count the number of zero-click days
#         zero_click_days = (filtered_df[filtered_df['date']<=n]['total_daily_clicks'] == 0).sum()

#         # Calculate the total number of days recorded for the student
#         total_days = filtered_df[filtered_df['date']<=n].shape[0]

#         # Compute the ratio of zero-click days
#         zero_day_ratio = zero_click_days / total_days

#         # Calculate the standard deviation of 'total_daily_clicks'
#         stdv = filtered_df[filtered_df['date']<=n]['total_daily_clicks'].std()
#         # Sum of 'total_daily_clicks' for this student
#         total_clicks = filtered_df['total_daily_clicks'].sum()

        # Append the student ID, mean clicks, zero_day_ratio, and standard deviation to the list
    
        data.append([student_id, code_module, code_presentation, mean_clicks, zero_day_ratio, stdv, total_clicks, final_result])

    # Create a new DataFrame from the list
    df_summary = pd.DataFrame(data, columns=['id_student','code_module','code_presentation', 'mean_clicks', 'zero_day_ratio', 'stdv','total_clicks','final_result'])
    return df_summary


In [6]:
#Create dataset with interaction predictors to hand off to Maysam
df_AAA_2013J_75 = df_interaction_n('AAA','2013J',75)
df_AAA_2013J_75.to_csv('df_AAA_2013J_75.csv', index=False)
df_AAA_2014J_75 = df_interaction_n('AAA','2014J',75)
df_AAA_2014J_75.to_csv('df_AAA_2014J_75.csv', index=False)
df_BBB_2013B_75 = df_interaction_n('BBB','2013B',75)
df_BBB_2013B_75.to_csv('df_BBB_2013B_75.csv', index=False)
df_BBB_2013J_75 = df_interaction_n('BBB','2013J',75)
df_BBB_2013J_75.to_csv('df_BBB_2013J_75.csv', index=False)
df_BBB_2014B_75 = df_interaction_n('BBB','2014B',75)
df_BBB_2014B_75.to_csv('df_BBB_2014B_75.csv', index=False)
df_BBB_2014J_75 = df_interaction_n('BBB','2014J',75)
df_BBB_2014J_75.to_csv('df_BBB_2014J_75.csv', index=False)
df_CCC_2014B_75 = df_interaction_n('CCC','2014B',75)
df_CCC_2014B_75.to_csv('df_CCC_2014B_75.csv', index=False)
df_CCC_2014J_75 = df_interaction_n('CCC','2014J',75)
df_CCC_2014J_75.to_csv('df_CCC_2014J_75.csv', index=False)
df_DDD_2013B_75 = df_interaction_n('DDD','2013B',75)
df_DDD_2013B_75.to_csv('df_DDD_2013B_75.csv', index=False)
df_DDD_2013J_75 = df_interaction_n('DDD','2013J',75)
df_DDD_2013J_75.to_csv('df_DDD_2013J_75.csv', index=False)
df_DDD_2014B_75 = df_interaction_n('DDD','2014B',75)
df_DDD_2014B_75.to_csv('df_DDD_2014B_75.csv', index=False)
df_DDD_2014J_75 = df_interaction_n('DDD','2014J',75)
df_DDD_2014J_75.to_csv('df_DDD_2014J_75.csv', index=False)
df_EEE_2013J_75 = df_interaction_n('EEE','2013J',75)
df_EEE_2013J_75.to_csv('df_EEE_2013J_75.csv', index=False)
df_EEE_2014B_75 = df_interaction_n('EEE','2014B',75)
df_EEE_2014B_75.to_csv('df_EEE_2014B_75.csv', index=False)
df_EEE_2014J_75 = df_interaction_n('EEE','2014J',75)
df_EEE_2014J_75.to_csv('df_EEE_2014J_75.csv', index=False)
df_FFF_2013B_75 = df_interaction_n('FFF','2013B',75)
df_FFF_2013B_75.to_csv('df_FFF_2013B_75.csv', index=False)
df_FFF_2013J_75 = df_interaction_n('FFF','2013J',75)
df_FFF_2013J_75.to_csv('df_FFF_2013J_75.csv', index=False)
df_FFF_2014B_75 = df_interaction_n('FFF','2014B',75)
df_FFF_2014B_75.to_csv('df_FFF_2014B_75.csv', index=False)
df_FFF_2014J_75 = df_interaction_n('FFF','2014J',75)
df_FFF_2014J_75.to_csv('df_FFF_2014J_75.csv', index=False)
df_GGG_2013J_75 = df_interaction_n('GGG','2013J',75)
df_GGG_2013J_75.to_csv('df_GGG_2013J_75.csv', index=False)
df_GGG_2014B_75 = df_interaction_n('GGG','2014B',75)
df_GGG_2014B_75.to_csv('df_GGG_2014B_75.csv', index=False)
df_GGG_2014J_75 = df_interaction_n('GGG','2014J',75)
df_GGG_2014J_75.to_csv('df_GGG_2014J_75.csv', index=False)

In [7]:
#combine all csv files in VLE Full Dataset folder into one big dataframe

import pandas as pd
import os

# Specify the directory containing the CSV files
directory = 'VLE Full Dataset'

# List to hold dataframes
dataframes = []

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Create the full path to the file
        filepath = os.path.join(directory, filename)
        # Read the CSV file and append the DataFrame to the list
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Concatenate all the dataframes in the list
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined dataframe
print(combined_df.head())

# # Optionally, save the combined DataFrame to a new CSV file
# combined_df.to_csv('combined.csv', index=False)


   id_student code_module code_presentation  mean_clicks  zero_day_ratio  \
0       11391         AAA             2013J     6.839506        0.753086   
1       28400         AAA             2013J     8.744186        0.627907   
2       30268         AAA             2013J    12.217391        0.478261   
3       31604         AAA             2013J    11.081395        0.441860   
4       32885         AAA             2013J     7.279070        0.639535   

        stdv  total_clicks final_result  
0  20.565418         554.0         Pass  
1  18.213884         752.0         Pass  
2  17.079910         281.0    Withdrawn  
3  16.066930         953.0         Pass  
4  15.853484         626.0         Pass  


Does the column contain None or NaN values? False


In [10]:
#convert pass and distinction to 1 and fail and withdrawn to 0
combined_df['final_result'] = np.where(combined_df['final_result'].isin(['Pass', 'Distinction']), 1, 0)

In [11]:
combined_df

,id_student,code_module,code_presentation,mean_clicks,zero_day_ratio,stdv,total_clicks,final_result
0,11391,AAA,2013J,6.839506,0.753086,20.565418,554.0,1
1,28400,AAA,2013J,8.744186,0.627907,18.213884,752.0,1
2,30268,AAA,2013J,12.217391,0.478261,17.079910,281.0,0
3,31604,AAA,2013J,11.081395,0.441860,16.066930,953.0,1
4,32885,AAA,2013J,7.279070,0.639535,15.853484,626.0,1
...,...,...,...,...,...,...,...,...
29223,2640965,GGG,2014J,1.708333,0.791667,4.685585,41.0,0
29224,2645731,GGG,2014J,1.421053,0.859649,5.681887,81.0,1
29225,2648187,GGG,2014J,0.406977,0.941860,2.008089,35.0,1
29226,2679821,GGG,2014J,2.966667,0.877778,11.431722,267.0,0


In [12]:
# Optionally, save the combined DataFrame to a new CSV file
combined_df.to_csv('vle_full_dataset.csv', index=False)
